In [ ]:
counties_of_interest={"Albany","Rensselaer","Greene","Ulster","Jefferson","Lewis","Saint Lawrence","Franklin","Clinton","Essex","Hamilton","Warren","Fulton","Saratoga","Washington","Montgomery","Schnectady","Schoharie","Dutchess","Rockland"}
  #,"Columbia"

counties = {}

counties["Albany"] = ["12159","12161"]

In [ ]:
#counties["Troy"]

In [ ]:
import requests
import numpy as np
from bs4 import BeautifulSoup

Let's get the data using [requests](http://docs.python-requests.org/en/master/) and [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/).

In [ ]:
for county in counties_of_interest:

  print(county)
  
  url = 'https://data.mongabay.com/igapo/zip_codes/counties/alpha/%s%sCounty-New%sYork1.html' % (county,"%20", "%20" )
  
  if county=='Saint Lawrence':
    url = 'https://data.mongabay.com/igapo/zip_codes/counties/alpha/Saint%20Lawrence%20County-New%20York1.html'
  
  print(url)
  
  r = requests.get(url, auth=('user', 'pass'))
  print(r.status_code)
  print(r.headers['content-type'])

  soup =  BeautifulSoup(r.text, 'html.parser')

  print(soup.title)
  
  vals = soup.find_all("td")

  zcodes = []

  for val in vals:

    x = val.get_text()

    zcode = 0

    if len(x)<5:
      continue

    try:
      zcode = int(x[0:5])
    except ValueError:
      #print(x)
      1

    if int(zcode)>1000:
      if zcode not in zcodes:
        zcodes.append(zcode)
        
  counties[county] = zcodes

  
print(counties['Albany'])

In [ ]:
keys = counties.keys()

for key in keys:
  zips = counties[key]
  
  print("key: %15s    # zips: %d" % (key,len(zips)))

In [ ]:
!pip install folium

!pip install zipcode

In [ ]:
from folium.plugins import MarkerCluster
import folium
import os


import zipcode


zcodes = counties["Albany"]
#print(zcodes)

colors = ['red','blue','green','black']

myzip = zipcode.isequal(str(zcodes[0]))

m = folium.Map(location=[myzip.lat, myzip.lon], zoom_start=8)
#marker_cluster = MarkerCluster().add_to(m)


for i,key in enumerate(counties.keys()):

  zcodes = counties[key]
  
  for zcode in zcodes:
    myzip = zipcode.isequal(str(zcode))
    #print(zcode, myzip.lat,myzip.lon)

    '''
    idx = 0
    while 1:
      if zcode not in zindices:
        zcode += 1
      else:
        idx = zindices.index(zcode)
        break

    lat = allzipcodes[1][idx]
    lon = allzipcodes[2][idx]

    '''
    lat = myzip.lat
    lon = myzip.lon

    folium.Marker(
      location=[lat, lon],
      popup=str(zcode),
      icon=folium.Icon(color=colors[i% len(colors)], icon='ok-sign'),
  ).add_to(m)


m.save('maptest.html')

m

In [ ]:
z = counties["Albany"]

print(z)

if 12211 in z:
  print("This is in the county!")
else:
  print("This is not in the county")

In [ ]:
x = zcodes[20]

In [ ]:
a = zipcode.isequal(str(x))

In [ ]:
a.to_dict()

In [ ]:
!wget 'https://gist.githubusercontent.com/erichurst/7882666/raw/5bdc46db47d9515269ab12ed6fb2850377fd869e/US%2520Zip%2520Codes%2520from%25202013%2520Government%2520Data'


In [ ]:
infilename = 'US%20Zip%20Codes%20from%202013%20Government%20Data'
allzipcodes = np.loadtxt(infilename,unpack=True,delimiter=',',skiprows=1)

In [ ]:
len(allzipcodes[0])

zindices = allzipcodes[0].astype(int).tolist()

In [ ]:
counties

In [ ]:
zipvals = np.loadtxt('zipcodes_coordinates.txt',skiprows=1,unpack=True,dtype=str,delimiter=',')

In [ ]:
keys = counties.keys()

for key in keys:
    county = counties[key]
    
    newinfo = {}
    
    for azipcode in counties[key]:
        
        zcode = azipcode
        
        found_zipcode=False
        while found_zipcode==False:
            z = str(zcode)
            try:
                idx = zipvals[0].tolist().index(z)
                found_zipcode = True
            except:
                print("Couldn't find it: ",zcode)
                zcode -= 1
            
        lat = float(zipvals[1][idx])
        lon = float(zipvals[2][idx])
        
        newinfo[str(azipcode)] = {}
        newinfo[str(azipcode)]["lat"] = lat
        newinfo[str(azipcode)]["lon"] = lon

        print(lat,lon)
    counties[key] = newinfo

In [ ]:
counties

In [ ]:
keys = counties.keys()

zipinfos = {}

for key in keys:
    
    zips = counties[key].keys()
    
    for z in zips:
        zipinfos[z] = key

In [ ]:
zipinfos

In [ ]:
import pickle

filename = "CARES_zipcode_data.pkl"
outfile = open(filename,'wb')

pickle.dump([counties,zipinfos],outfile,1)
#pickle.dump(zipinfos,outfile,1)
outfile.close()


In [ ]:
!ls -ltr

In [ ]:
c,z = pickle.load(open('CARES_zipcode_data.pkl','rb'),encoding='latin')

In [ ]:
z